* Problem Statement : Wild fires are huge loss of environment and economy, this disasture had a huge impact on humans and animals. If the wild chances of wild fires are predicted then some steps can be taken in order to stop wild fires from causing much heavy loss.
* Aim : Aim is based on the physical factors (Rain, Temp, RH) predict whether there any chance of wild fire or not.

In [2]:
from data.interim import data
import pandas as pd
import numpy as np

In [3]:
data.head()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
0,1,6,2012,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,not fire
1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,not fire
2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire
3,4,6,2012,25,89,13,2.5,28.6,1.3,6.9,0.0,1.7,0.0,not fire
4,5,6,2012,27,77,16,0.0,64.8,3.0,14.2,1.2,3.9,0.5,not fire


Changing the values in Classes to numeric

In [5]:
data.Classes = data.Classes.map({'not fire':1, 'fire':0})

In [4]:
data.head()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
0,1,6,2012,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,not fire
1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,not fire
2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire
3,4,6,2012,25,89,13,2.5,28.6,1.3,6.9,0.0,1.7,0.0,not fire
4,5,6,2012,27,77,16,0.0,64.8,3.0,14.2,1.2,3.9,0.5,not fire


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

Droping the Year column as it is same in whole dataset.

In [6]:
data.drop(columns=['year'], inplace=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          244 non-null    int64  
 1   month        244 non-null    int64  
 2   Temperature  244 non-null    int64  
 3   RH           244 non-null    int64  
 4   Ws           244 non-null    int64  
 5   Rain         244 non-null    float64
 6   FFMC         244 non-null    float64
 7   DMC          244 non-null    float64
 8   DC           244 non-null    float64
 9   ISI          244 non-null    float64
 10  BUI          244 non-null    float64
 11  FWI          244 non-null    float64
 12  Classes      244 non-null    int64  
dtypes: float64(7), int64(6)
memory usage: 24.9 KB


In [7]:
feature, target = data.drop(columns=['Classes']), data.Classes

In [8]:
nums_pipe = Pipeline(
    steps = (
        ('impute', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
)
)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature, target, random_state=7, test_size=0.25)

In [10]:
nums_pipe.fit(X_train)
X_train = nums_pipe.transform(X_train)
X_test = nums_pipe.transform(X_test)

In [11]:
X_test

array([[ 5.12675497e-01,  4.56700835e-01,  7.88045485e-01,
         4.14659520e-01,  1.29861023e+00, -3.89022181e-01,
         7.57554416e-01,  1.02870916e+00,  1.93291008e+00,
         9.88718107e-01,  1.44275235e+00,  1.42186877e+00],
       [-8.67024738e-01, -1.36017423e+00, -1.19133935e+00,
        -7.15932739e-01,  5.74080743e-01,  2.51008088e-01,
        -4.48007251e-01, -5.75175649e-01, -8.89301620e-01,
        -7.48985509e-01, -6.69128786e-01, -8.27189500e-01],
       [-1.32692482e+00, -1.36017423e+00, -2.03964714e+00,
         1.81127349e+00, -8.74978237e-01,  8.41805259e-01,
        -3.50114567e+00, -1.13917910e+00, -9.00755401e-01,
        -1.10604790e+00, -1.10075413e+00, -9.39642413e-01],
       [-1.55687486e+00,  4.56700835e-01,  2.20189180e+00,
        -1.84652500e+00, -5.12713492e-01, -3.89022181e-01,
         1.11422947e+00, -3.01986478e-01, -5.68595745e-01,
         2.17892606e+00, -4.14778139e-01,  9.58000505e-01],
       [ 1.20252562e+00, -4.51736696e-01,  7.8804548

In [19]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [15]:
model = DecisionTreeClassifier()
parameters = dict(
    criterion=['gini', 'entropy'],
    max_depth=[5, 8, 10, 15, 3],
    min_samples_split=[2, 3, 4],
)

clf = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    cv=4,
    verbose=1,
    scoring='accuracy'
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

Fitting 4 folds for each of 30 candidates, totalling 120 fits


In [16]:
clf.best_score_

0.9781400966183575

In [18]:
clf.best_estimator_

DecisionTreeClassifier(max_depth=15, min_samples_split=3)

In [20]:
model = ExtraTreesClassifier()
parameters = dict(
    n_estimators=[100, 150, 180, 200],
    criterion=['gini', 'entropy'],
    max_depth=[5, 8, 10, 15, 3],
    min_samples_split=[2, 3, 4],
)

clf = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    cv=4,
    verbose=1
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

Fitting 4 folds for each of 120 candidates, totalling 480 fits


In [23]:
clf.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'min_samples_split': 2,
 'n_estimators': 100}

In [22]:
clf.best_score_

0.9670289855072465

In [24]:
model = RandomForestClassifier()
parameters = dict(
    n_estimators=[100, 150, 180, 200],
    criterion=['gini', 'entropy'],
    max_depth=[5, 8, 10, 15, 3],
    min_samples_split=[2, 3, 4],
)

clf = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    cv=4,
    verbose=1,
    scoring='accuracy'
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

Fitting 4 folds for each of 120 candidates, totalling 480 fits


In [25]:
clf.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'min_samples_split': 3,
 'n_estimators': 200}

In [26]:
clf.best_score_

0.9834541062801933

In [27]:
model = SVC()
parameters = dict(
    degree=[2, 3, 4, 5],
    kernel=['linear', 'poly', 'rbf'],
    gamma=['scale', 'auto'],
    C=[1, 2, 3]
)
clf = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring='accuracy',
    cv=4,
    verbose=1
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

Fitting 4 folds for each of 72 candidates, totalling 288 fits


In [28]:
clf.best_params_

{'C': 2, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}

In [29]:
clf.best_score_

0.9782608695652174

So out of all the algorithms Random Forest perform very well with a accuracy of 98.34%